## Experiment Setup

In [1]:
import torch
import numpy as np

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# CUDA Setup
if torch.cuda.is_available():
    use_gpu = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Dataset to use
dataset_name = 'yacht'

# Training set size
dataset_train_size = 0.8

# Epochs
n_epochs = 100

## Prepare data

### Get the data as a torch Dataset object

In [2]:
from ronald_bdl import datasets

if dataset_name == 'MNIST':
    dataset = datasets.MNIST(root_dir='./datasets_files', transform=None, download=True)
else:
    dataset = datasets.UCIDatasets(dataset_name, root_dir='./datasets_files', transform=None, download=True)

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data


### Split into training/test set using torch's DataLoader

In [3]:
from torch.utils.data import random_split, DataLoader

train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size

train, test = random_split(dataset, lengths=[train_size, test_size])

train_loader = DataLoader(train)
test_loader = DataLoader(test)

## Define network

In [4]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=len(dataset.features), 
    output_dim=len(dataset.targets),
    hidden_dim=50,
    n_hidden=1,
    n_predictions=10000,
    dropout_rate=0.01,
)

# Print the network structure
print(network)

FCNetMCDropout(
  (input): Linear(in_features=6, out_features=50, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
)


## Train the network

### Setup optimizer

In [5]:
from torch import nn, optim

objective = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=0.01)

### Run the optimizer

In [6]:
# Partially adapted from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
for epoch in range(n_epochs): # loop over the dataset multiple times
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(inputs)

        loss = objective(outputs, targets)
        loss.backward()

        optimizer.step()

## Make predictions

In [8]:
for i, data in enumerate(test_loader):
    inputs, targets = data
    
    predictions, mean, var = network.mc_predict(inputs)

    print("Test " + str(i))
    print("Original prediction = " + str(network(inputs)))
    print("Mean = " + str(mean))
    print("Variance = " + str(var))

Test 0
Original prediction = tensor([[6.7675]], grad_fn=<AddmmBackward>)
Mean = tensor([6.6605], grad_fn=<MeanBackward1>)
Variance = tensor([0.3123], grad_fn=<VarBackward1>)
Test 1
Original prediction = tensor([[1.4650]], grad_fn=<AddmmBackward>)
Mean = tensor([1.4596], grad_fn=<MeanBackward1>)
Variance = tensor([0.0006], grad_fn=<VarBackward1>)
Test 2
Original prediction = tensor([[2.6716]], grad_fn=<AddmmBackward>)
Mean = tensor([2.6434], grad_fn=<MeanBackward1>)
Variance = tensor([0.0227], grad_fn=<VarBackward1>)
Test 3
Original prediction = tensor([[18.5262]], grad_fn=<AddmmBackward>)
Mean = tensor([18.1966], grad_fn=<MeanBackward1>)
Variance = tensor([2.1241], grad_fn=<VarBackward1>)
Test 4
Original prediction = tensor([[1.3071]], grad_fn=<AddmmBackward>)
Mean = tensor([1.3045], grad_fn=<MeanBackward1>)
Variance = tensor([0.0001], grad_fn=<VarBackward1>)
Test 5
Original prediction = tensor([[1.3071]], grad_fn=<AddmmBackward>)
Mean = tensor([1.3047], grad_fn=<MeanBackward1>)
Varian

Test 47
Original prediction = tensor([[1.3071]], grad_fn=<AddmmBackward>)
Mean = tensor([1.3047], grad_fn=<MeanBackward1>)
Variance = tensor([0.0001], grad_fn=<VarBackward1>)
Test 48
Original prediction = tensor([[1.3071]], grad_fn=<AddmmBackward>)
Mean = tensor([1.3045], grad_fn=<MeanBackward1>)
Variance = tensor([0.0001], grad_fn=<VarBackward1>)
Test 49
Original prediction = tensor([[21.7182]], grad_fn=<AddmmBackward>)
Mean = tensor([21.3168], grad_fn=<MeanBackward1>)
Variance = tensor([2.7489], grad_fn=<VarBackward1>)
Test 50
Original prediction = tensor([[6.2145]], grad_fn=<AddmmBackward>)
Mean = tensor([6.1119], grad_fn=<MeanBackward1>)
Variance = tensor([0.2785], grad_fn=<VarBackward1>)
Test 51
Original prediction = tensor([[28.2015]], grad_fn=<AddmmBackward>)
Mean = tensor([27.6055], grad_fn=<MeanBackward1>)
Variance = tensor([5.6438], grad_fn=<VarBackward1>)
Test 52
Original prediction = tensor([[1.3071]], grad_fn=<AddmmBackward>)
Mean = tensor([1.3045], grad_fn=<MeanBackward1>